<a href="https://colab.research.google.com/github/georgemoum-cylearn/gmlearn/blob/main/pcap_data_IPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#εισαγωγή βιβλιοθηκών - πακέτων
import matplotlib.pyplot as plt
from  tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score,  f1_score, confusion_matrix
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [15]:
#φόρτωση του συνόλου δεδομένων
url = "https://raw.githubusercontent.com/jimtak80/AI_EKPA/main/Data/pcap_data.csv"
df = pd.read_csv(url)


In [16]:
#δημιουργία του συνόλου σε δύο κλάσεις Normal (0) και Attack (1)
X = df.drop("target", axis=1)
y = df["target"].apply(lambda x: 0 if x == 0.0 else 1)

In [17]:
#έλεγχος κλάσεων
print("Κατανομή κλάσεων y:")
print(y.value_counts())

Κατανομή κλάσεων y:
target
1    9965
0    5002
Name: count, dtype: int64


In [19]:
#διαχωρισμός κατηγορικών και αριθμητικών μεταβλητών
numeric_features = X.columns.difference(categorical_features)

In [20]:
#δημιουργία διοχέτευσης (αγωγού) προεπεξεργασίας με κωδικοποίηση μίας δέσμης (one-hot encoding) για κατηγορικές μεταβλητές
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [21]:
#τμηματοποίηση δεδομένων σε σύνολα εκαπίδευσης και δοκιμών
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print("Κατανομή κλάσεων y_train μετά τον διαχωρισμό:")
print(y_train.value_counts())

Κατανομή κλάσεων y_train μετά τον διαχωρισμό:
target
1    6975
0    3501
Name: count, dtype: int64


In [22]:
#δημιουργία διοχέτευσης (αγωγού) SMOTE για τις αριθμητικές μεταβλητές μόνο
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [23]:
#καθορισμός κατωφλίου (threshold) για την διακοπή διακτυακής κίνησης
blocking_threshold = 0.9

In [24]:
#δημιουργία βρόχου συνεχούς - αυξητικής μάθησης
batch_size = 1000

# Shuffle X_train and y_train together to ensure mixed batches
shuffled_indices = np.random.permutation(len(X_train))
X_train_shuffled = X_train.iloc[shuffled_indices].reset_index(drop=True)
y_train_shuffled = y_train.iloc[shuffled_indices].reset_index(drop=True)

for epoch in range(1, 3):  #δυνατότητα αλλαγής των εποχών
    for i in range(0, len(X_train_shuffled), batch_size):
        X_batch = X_train_shuffled.iloc[i:i + batch_size]
        y_batch = y_train_shuffled.iloc[i:i + batch_size]

        # Ελέγξτε αν το y_batch έχει περισσότερες από μία κλάσεις
        if len(y_batch.unique()) < 2:
            print(f"Skipping batch {i} due to single class in y_batch.")
            continue

        #σταδιακή ενημέρωση του μοντέλου με κάθε ροή (batch) δεδομένων
        pipeline.fit(X_batch, y_batch)

        #περιοδική ενημέρωση του μοντέλου στο σύνολο δοκιμών
        #ΠΡΟΣΟΧΗ: Αυτή η συνθήκη ίσως χρειαστεί προσαρμογή για να ταιριάζει με τον αριθμό των batches
        #Για παράδειγμα, αν θέλετε να αξιολογήσετε μετά από κάθε batch, αφαιρέστε το `and i > 0`
        if i % batch_size == 0 and i > 0:
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

            #αποκλεισμός δικτυακής κυκλοφορίας εάν η προβλεπόμενη πιθανότητα υπερβαίνει το καθορισμένο όριο
            blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
            if len(blocked_indices) > 0:
                print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

            accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
            print(f"Epoch {epoch}, Iteration {i}, Test Accuracy: {accuracy}")

ValueError: A given column is not a column of the dataframe

In [ ]:
#τελική αξιολόγηση του μοντέλου στο σύνολο δοκιμών
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
if len(blocked_indices) > 0:
    print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
classification_rep = classification_report(y_test, y_pred_proba > blocking_threshold)

#εκτύπωση τελικών αποτελεσμάτων
print(f"Final Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Blocking 530 malicious traffic instances.
Final Test Accuracy: 0.44333110665776
Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.99      0.54      1501
           1       0.96      0.17      0.29      2990

    accuracy                           0.44      4491
   macro avg       0.67      0.58      0.42      4491
weighted avg       0.77      0.44      0.37      4491

